In [ ]:
"""SELECT a.`user_id`,
       case when TO_DAYS(h.`register_time`)= TO_DAYS(a.`draw_time`) then 1 else 0 end as '是否新用户', 
       case when a.`draw_type`= 1 then '直接领取' when a.`draw_type`= 2 then '主持人转发' when a.`draw_type`= 3 then '微信' else a.`draw_type` end as '领取方式',
       a.`coupon_detailId`, 
       c.`coupon_name` as '米票名称',
       d.`productName` as '商品名称',
       a.`draw_qty` as '领取数量', 
       a.`draw_time` as '领取时间', 
       a.`ref_programId` as '领取社区id', 
       e.`name` as '领取社区', 
       case when a.`status`= 0 then '待使用' when a.`status`= 1 then '已使用' else a.`status` end as '使用状态',
       a.`ref_orderId`, f.`programId` as '订单社区id', g.`name` as '订单社区', f.`create_time` as '订单生成时间',
       aj.`orderStatus` ,
       case when aj.`orderStatus`in (1,4,5,6,7,10) then 1 else 0 end as ind_cover
  FROM `ajmide_main_m`.`aj_coupon_draw_list` a
  LEFT JOIN `aj_coupon_detail` b on a.`coupon_detailId`= b.`seq_id`
  LEFT JOIN `aj_coupon` c on b.`coupon_id`= c.`coupon_id`
  LEFT JOIN `aj_product` d on a.`ref_productId`= d.`id`
  LEFT JOIN `program` e on a.`ref_programId`= e.`program_id`
  LEFT JOIN `aj_order` f on a.`ref_orderId`= f.`id`
  LEFT JOIN `program` g on f.`programId`= g.`program_id`
  LEFT JOIN `user_info` h on a.`user_id`= h.`user_id`
  left join `aj_order`  aj on  a.`ref_orderId` = aj.`id`
 where a.`draw_time` between '2018-01-19'
   and '2018-01-23 19:59:59'"""

In [2]:
import pandas as pd
from pandasql import sqldf
pd.set_option('display.height',2000)

height has been deprecated.



In [3]:
df = pd.read_csv('/Users/admin/Downloads/sqlResult_1649470.csv', header = 0)


In [4]:
df.head()

,user_id,是否新用户,领取方式,coupon_detailId,米票名称,商品名称,领取数量,领取时间,领取社区id,领取社区,使用状态,ref_orderId,订单社区id,订单社区,订单生成时间,orderStatus,ind_cover,领取日期,订单日期
0,1974282,0,直接领取,5,云纯挂耳咖啡米票,【年货节】云纯挂耳式滤泡咖啡,1,2018-01-19 05:36:11,0,NaN,待使用,0,NaN,NaN,NaN,NaN,0,2018-01-19,NaN
1,3513156,0,直接领取,2,520老母鸡套餐米票,【年货节】2只纯正520老母鸡+30枚徐鸿飞小鲜蛋,1,2018-01-19 05:53:24,0,NaN,待使用,0,NaN,NaN,NaN,NaN,0,2018-01-19,NaN
2,3513156,0,直接领取,4,早康枸杞米票,【年货节】早康上品有机枸杞袋装180g,1,2018-01-19 05:54:06,0,NaN,待使用,0,NaN,NaN,NaN,NaN,0,2018-01-19,NaN
3,168063,0,直接领取,2,520老母鸡套餐米票,【年货节】2只纯正520老母鸡+30枚徐鸿飞小鲜蛋,1,2018-01-19 06:06:07,0,NaN,待使用,0,NaN,NaN,NaN,NaN,0,2018-01-19,NaN
4,1724713,0,直接领取,5,云纯挂耳咖啡米票,【年货节】云纯挂耳式滤泡咖啡,1,2018-01-19 06:12:49,0,NaN,待使用,0,NaN,NaN,NaN,NaN,0,2018-01-19,NaN


In [5]:
q="""
    select 
       date("领取时间") as date_ind,
       count(distinct user_id) as "总用户量",
       sum(1) as "总用户次数",
       count(distinct case when "使用状态"="已使用" and ind_cover =1  then user_id end) as "总使用用户数",
       sum( case when "使用状态"="已使用" and ind_cover =1  then 1 end) as "总使用用户次数"
    from 
       df 
    group by
       date("领取时间")
       """


In [6]:
new_df = sqldf(q)
new_df

,date_ind,总用户量,总用户次数,总使用用户数,总使用用户次数
0,2018-01-19,224,785,7,13
1,2018-01-20,656,2972,10,13
2,2018-01-21,2822,12912,19,23
3,2018-01-22,1206,6262,23,31
4,2018-01-23,1419,5158,25,29
5,2018-01-24,1791,4809,20,21
6,2018-01-25,607,1586,22,29
7,2018-01-26,480,1010,11,13


In [7]:
q="""
    select
       date("领取时间") as date_ind,
       "领取方式",
       count(distinct user_id) as "总用户量",
       sum(1) as "总用户次数",
       count(distinct case when "使用状态"="已使用" and ind_cover =1  then user_id end) as "总使用用户数",
       sum( case when "使用状态"="已使用" and ind_cover =1  then 1 end) as "总使用用户次数"
    from 
       df
    group by
       date("领取时间"),
       "领取方式"
       """

In [8]:
new_df = sqldf(q)
new_df

,date_ind,领取方式,总用户量,总用户次数,总使用用户数,总使用用户次数
0,2018-01-19,主持人转发,26,134,3,4.0
1,2018-01-19,微信,89,333,3,5.0
2,2018-01-19,直接领取,118,318,3,4.0
3,2018-01-20,主持人转发,69,341,2,3.0
4,2018-01-20,微信,369,1502,2,2.0
5,2018-01-20,直接领取,225,1129,6,8.0
6,2018-01-21,主持人转发,45,225,0,NaN
7,2018-01-21,微信,1433,5817,10,11.0
8,2018-01-21,直接领取,1354,6870,9,12.0
9,2018-01-22,主持人转发,510,2595,9,10.0


In [11]:
q="""
    select
       date("领取时间") as date_ind,
       "领取社区",
       count(distinct user_id) as "总用户量",
       sum(1) as "总用户次数",
       count(distinct case when "使用状态"="已使用"   and ind_cover =1 then user_id end) as "总使用用户数",
       sum( case when "使用状态"="已使用"  and ind_cover =1  then 1 end) as "总使用用户次数"
    from 
       df
    group by
       date("领取时间"),
       "领取社区"
       """

In [12]:
new_df = sqldf(q)
new_df

,date_ind,领取社区,总用户量,总用户次数,总使用用户数,总使用用户次数
0,2018-01-19,None,206,651,6,9.0
1,2018-01-19,动感101特别专区,3,15,0,NaN
2,2018-01-19,早安新发现,4,20,0,NaN
3,2018-01-19,最爱K歌,2,10,0,NaN
4,2018-01-19,欢乐夜高峰·晚安上海,2,10,0,NaN
5,2018-01-19,活到100岁,1,5,0,NaN
6,2018-01-19,活动进行时,2,10,0,NaN
7,2018-01-19,海呱子,1,1,0,NaN
8,2018-01-19,笑声与歌声,1,5,0,NaN
9,2018-01-19,股市大家谈,1,5,0,NaN


In [9]:
q="""
    select
       date("领取时间") as date_ind,
       "商品名称",
       count(distinct user_id) as "总用户量",
       sum(1) as "总用户次数",
       count(distinct case when "使用状态"="已使用" and "订单状态"!=6 then user_id end) as "总使用用户数",
       sum( case when "使用状态"="已使用" and "订单状态"!=6 then 1 end) as "总使用用户次数"
    from 
       df
    group by
       date("领取时间"),
       "商品名称"
       """

In [10]:

new_df = sqldf(q)
new_df

,date_ind,商品名称,总用户量,总用户次数,总使用用户数,总使用用户次数
0,2018-01-19,Beelab 碧罗兰 麦卢卡蜂蜜成人牙膏 100克/支,31,31,0,NaN
1,2018-01-19,TOMONI多功能干燥暖被机AFS-W9010,2,2,0,NaN
2,2018-01-19,idMASK防雾霾口罩轻装版,1,1,0,NaN
3,2018-01-19,【年货节】2只纯正520老母鸡+30枚徐鸿飞小鲜蛋,107,107,0,NaN
4,2018-01-19,【年货节】云纯挂耳式滤泡咖啡,132,132,2,2.0
5,2018-01-19,【年货节】仙缘海鲜大礼包,58,58,3,3.0
6,2018-01-19,【年货节】休闲零食年货大礼包,48,48,2,2.0
7,2018-01-19,【年货节】健康大礼包,58,58,1,1.0
8,2018-01-19,【年货节】年货大礼包,62,62,2,2.0
9,2018-01-19,【年货节】早康上品有机枸杞袋装180g,59,59,0,NaN


In [15]:
q="""
    select
       date("领取时间") as date_ind,
       "领取社区",
       "订单社区",
       count(distinct user_id) as "总用户量",
       sum(1) as "总用户次数",
       count(distinct case when "使用状态"="已使用"   and ind_cover =1 then user_id end) as "总使用用户数",
       sum( case when "使用状态"="已使用"  and ind_cover =1  then 1 end) as "总使用用户次数"
    from 
       df
    group by
       date("领取时间"),
       "领取社区",
       "订单社区"
       """

In [16]:
new_df = sqldf(q)
new_df

,date_ind,领取社区,订单社区,总用户量,总用户次数,总使用用户数,总使用用户次数
0,2018-01-19,None,None,204,641,0,NaN
1,2018-01-19,None,城市夜声,2,2,2,2.0
2,2018-01-19,None,早安新发现,3,3,3,3.0
3,2018-01-19,None,活到100岁,2,2,2,2.0
4,2018-01-19,None,活动进行时,1,1,0,NaN
5,2018-01-19,None,海呱子,1,1,1,1.0
6,2018-01-19,None,食在有味道,1,1,1,1.0
7,2018-01-19,动感101特别专区,None,3,15,0,NaN
8,2018-01-19,早安新发现,None,4,20,0,NaN
9,2018-01-19,最爱K歌,None,2,10,0,NaN


In [23]:
q="""

select 
 sum(1) as "领取数量",
 sum( case when "使用状态"="已使用" then 1 end) as "使用数量",
 count(distinct user_id) as "领取人数",
 count(distinct case when "使用状态"="已使用" then user_id end) as "使用人数"
from
 df
where 
  "领取时间">'2018-01-19'
"""

In [24]:
new_df = sqldf(q)
new_df

,领取数量,使用数量,领取人数,使用人数
0,35494,175,8777,132
